## Referências.
- [Documentação oficial da API do Spotify](https://developer.spotify.com/documentation/web-api)
- [Documentação oficial do Spotipy, uma biblioteca python para acessar a API](https://spotipy.readthedocs.io/en/latest/#)
    - [Tutorial de primeiros passos com o Spotipy](https://medium.com/@maxtingle/getting-started-with-spotifys-api-spotipy-197c3dc6353b)

## Primeiros passos (ambiente linux)
1. Crie um ambiente virtual. Na pasta atual, rode no terminal: `make venv`
2. Ative o ambiente virtual: `source spot_env/bin/activate`
3. Instale as dependências: `pip install -r requirements.txt`
4. Crie uma pasta com as variáveis de ambiente:
    - `touch .env`
    - Configure as variáveis `SPOTIPY_CLIENT_ID` e `SPOTIPY_CLIENT_SECRET` com os valores disponíveis no app criado no [site do spotify para desenvolvedores](https://developer.spotify.com/dashboard). Não as compartilhe nem use-as diretamente em código público. 


In [57]:
# carrega as variáveis de ambiente
from dotenv import load_dotenv
load_dotenv()

import os
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [58]:
client_credentials_manager = SpotifyClientCredentials(
    client_id=os.environ.get('SPOTIPY_CLIENT_ID'),
    client_secret=os.environ.get('SPOTIPY_CLIENT_SECRET')
)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [59]:
playlists = {
    'Global': '37i9dQZEVXbNG2KDcFcKOF',
    'Brazil': '37i9dQZEVXbKzoK95AbRy9',
    'Australia': '37i9dQZEVXbK4fwx2r07XW',
    'Austria': '37i9dQZEVXbM1EaZ0igDlz',
    'Argentina': '37i9dQZEVXbKPTKrnFPD0G',
    'Mexico': '37i9dQZEVXbKUoIkUXteF6',
    'USA': '37i9dQZEVXbLp5XoPON0wI'
}

In [60]:
df = pd.read_csv('top_songs.csv')
set(df['playlist_id'])

{'37i9dQZEVXbK4fwx2r07XW',
 '37i9dQZEVXbKPTKrnFPD0G',
 '37i9dQZEVXbKzoK95AbRy9',
 '37i9dQZEVXbLp5XoPON0wI',
 '37i9dQZEVXbM1EaZ0igDlz',
 '37i9dQZEVXbNG2KDcFcKOF'}

In [61]:
'''
Recebe um dicionário com o país e o id da sua respectiva playlist.
Retorna um DataFrame com as características de cada track.
'''
def playlists_into_df(playlists_dict: dict, df: pd.DataFrame) -> pd.DataFrame:
    country_list = []
    playlist_id_list = []
    artist_name = []
    track_name = []
    release_date = []
    popularity = []
    track_id = []
    for country, playlist_id in playlists_dict.items():
        # Faz uma requisição na API
        if playlist_id in set(df['playlist_id']):
            print(playlist_id)
            continue
        playlist = sp.playlist_items(
            playlist_id=playlist_id,
            fields='items.track(album.release_date, artists.name, name, popularity, id)'
        )
        tracks = playlist['items']
        for track in tracks:
            track = track['track']
            country_list.append(country)
            playlist_id_list.append(playlist_id)
            artist_name.append(track['artists'][0]['name'])
            track_name.append(track['name'])
            release_date.append(track['album']['release_date'])
            popularity.append(track['popularity'])
            track_id.append(track['id'])
    new_df = pd.DataFrame({
        'playlist_country': country_list,
        'artist_name': artist_name,
        'track_name': track_name,
        'release_date': release_date,
        'popularity': popularity,
        'track_id': track_id,
        'playlist_id': playlist_id_list
    })
    return pd.concat([df, new_df], ignore_index=True)

In [62]:
top_songs = playlists_into_df(playlists, df)

37i9dQZEVXbNG2KDcFcKOF
37i9dQZEVXbKzoK95AbRy9
37i9dQZEVXbK4fwx2r07XW
37i9dQZEVXbM1EaZ0igDlz
37i9dQZEVXbKPTKrnFPD0G
37i9dQZEVXbLp5XoPON0wI


In [63]:
top_songs

,playlist_country,artist_name,track_name,release_date,popularity,track_id,playlist_id
0,Global,Artemas,i like the way you kiss me,2024-03-19,97,2GxrNKugF82CnoRFbQfzPf,37i9dQZEVXbNG2KDcFcKOF
1,Global,Hozier,Too Sweet,2024-03-22,86,0AjmK0Eai4zGrLaJwPvrDp,37i9dQZEVXbNG2KDcFcKOF
2,Global,Benson Boone,Beautiful Things,2024-04-05,82,3xkHsmpQCBMytMJNiDf3Ii,37i9dQZEVXbNG2KDcFcKOF
3,Global,Djo,End of Beginning,2022-09-16,99,3qhlB30KknSejmIvZZLjOD,37i9dQZEVXbNG2KDcFcKOF
4,Global,Ariana Grande,we can't be friends (wait for your love),2024-03-08,95,51ZQ1vr10ffzbwIjDCwqm4,37i9dQZEVXbNG2KDcFcKOF
...,...,...,...,...,...,...,...
345,Mexico,Gabito Ballesteros,A Puro Dolor,2023-11-30,82,1rXxK89X0XGpIdTB6CwiFu,37i9dQZEVXbKUoIkUXteF6
346,Mexico,Alfredo Olivas,Vete,2022-09-30,81,1GtIK8iINes4gNt06F60NU,37i9dQZEVXbKUoIkUXteF6
347,Mexico,Peso Pluma,BELLAKEO,2023-12-07,89,05WVKTdZhlIMX4qqMLuo0f,37i9dQZEVXbKUoIkUXteF6
348,Mexico,Peso Pluma,Rompe La Dompe,2023-12-28,85,2OWv3lRR8PtOVuHWzQNRs2,37i9dQZEVXbKUoIkUXteF6


In [64]:
# Salva o df em um csv
top_songs.to_csv('top_songs.csv', index=False)

In [152]:
# Requisição para conseguir as features de cada track
# track_features = sp.audio_features(tracks=tsg_df['track_id'])


In [213]:
def save_track_features_to_df(track_features_list: list) -> pd.DataFrame:
    df_list = []

    # cria uma lista de dataframes com os dicts com as features de cada track
    for track_dict in track_features_list:
        df_list.append(pd.DataFrame([track_dict]))
    return pd.concat(df_list, ignore_index=True)


In [215]:
# Salva o df em um csv
df = save_track_features_to_df(track_features_list=track_features)
df.to_csv('top_songs_global_features.csv', index=False)

In [216]:
playlist_df = pd.read_csv('top_songs_global.csv')
playlist_df.head(10).head(10)

,playlist_name,artist_name,track_name,popularity,track_id
0,Top Songs - Global,Benson Boone,Beautiful Things,100,6tNQ70jh4OwmPGpYy6R2o9
1,Top Songs - Global,Ariana Grande,we can't be friends (wait for your love),94,51ZQ1vr10ffzbwIjDCwqm4
2,Top Songs - Global,Djo,End of Beginning,98,3qhlB30KknSejmIvZZLjOD
3,Top Songs - Global,Future,Like That,92,2tudvzsrR56uom6smgOcSf
4,Top Songs - Global,¥$,CARNIVAL,96,3w0w2T288dec0mgeZZqoNN
5,Top Songs - Global,Teddy Swims,Lose Control,93,17phhZDn6oGtzMe56NuWvj
6,Top Songs - Global,Tate McRae,greedy,96,3rUGC1vUpkDG9CZFHMur1t
7,Top Songs - Global,Hozier,Too Sweet,51,0AjmK0Eai4zGrLaJwPvrDp
8,Top Songs - Global,FloyyMenor,Gata Only,94,6XjDF6nds4DE2BBbagZol6
9,Top Songs - Global,Taylor Swift,Cruel Summer,96,1BxfuPKGuaTgP7aM0Bbdwr


In [217]:
features_df = pd.read_csv('top_songs_global_features.csv')
features_df.head(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.472,0.471,10,-5.692,1,0.0603,0.15100,0.000000,0.1400,0.219,105.029,audio_features,6tNQ70jh4OwmPGpYy6R2o9,spotify:track:6tNQ70jh4OwmPGpYy6R2o9,https://api.spotify.com/v1/tracks/6tNQ70jh4Owm...,https://api.spotify.com/v1/audio-analysis/6tNQ...,180304,3
1,0.645,0.663,5,-8.305,1,0.0447,0.06570,0.000032,0.0751,0.287,115.830,audio_features,51ZQ1vr10ffzbwIjDCwqm4,spotify:track:51ZQ1vr10ffzbwIjDCwqm4,https://api.spotify.com/v1/tracks/51ZQ1vr10ffz...,https://api.spotify.com/v1/audio-analysis/51ZQ...,228639,4
2,0.689,0.454,2,-7.643,1,0.0584,0.03510,0.002590,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246,4
3,0.814,0.676,11,-4.670,0,0.2310,0.00709,0.000013,0.1190,0.312,162.012,audio_features,2tudvzsrR56uom6smgOcSf,spotify:track:2tudvzsrR56uom6smgOcSf,https://api.spotify.com/v1/tracks/2tudvzsrR56u...,https://api.spotify.com/v1/audio-analysis/2tud...,267707,4
4,0.594,0.811,1,-5.746,1,0.1590,0.18900,0.000000,0.3390,0.311,148.144,audio_features,3w0w2T288dec0mgeZZqoNN,spotify:track:3w0w2T288dec0mgeZZqoNN,https://api.spotify.com/v1/tracks/3w0w2T288dec...,https://api.spotify.com/v1/audio-analysis/3w0w...,264324,4
5,0.561,0.604,9,-4.409,1,0.0337,0.19900,0.000019,0.1040,0.242,159.920,audio_features,17phhZDn6oGtzMe56NuWvj,spotify:track:17phhZDn6oGtzMe56NuWvj,https://api.spotify.com/v1/tracks/17phhZDn6oGt...,https://api.spotify.com/v1/audio-analysis/17ph...,210689,3
6,0.750,0.733,6,-3.180,0,0.0319,0.25600,0.000000,0.1140,0.844,111.018,audio_features,3rUGC1vUpkDG9CZFHMur1t,spotify:track:3rUGC1vUpkDG9CZFHMur1t,https://api.spotify.com/v1/tracks/3rUGC1vUpkDG...,https://api.spotify.com/v1/audio-analysis/3rUG...,131872,1
7,0.741,0.620,10,-5.505,1,0.0412,0.02950,0.000809,0.0398,0.934,117.038,audio_features,0AjmK0Eai4zGrLaJwPvrDp,spotify:track:0AjmK0Eai4zGrLaJwPvrDp,https://api.spotify.com/v1/tracks/0AjmK0Eai4zG...,https://api.spotify.com/v1/audio-analysis/0Ajm...,251424,4
8,0.791,0.499,8,-8.472,0,0.0509,0.44600,0.000024,0.0899,0.669,99.986,audio_features,6XjDF6nds4DE2BBbagZol6,spotify:track:6XjDF6nds4DE2BBbagZol6,https://api.spotify.com/v1/tracks/6XjDF6nds4DE...,https://api.spotify.com/v1/audio-analysis/6XjD...,222000,4
9,0.552,0.702,9,-5.707,1,0.1570,0.11700,0.000021,0.1050,0.564,169.994,audio_features,1BxfuPKGuaTgP7aM0Bbdwr,spotify:track:1BxfuPKGuaTgP7aM0Bbdwr,https://api.spotify.com/v1/tracks/1BxfuPKGuaTg...,https://api.spotify.com/v1/audio-analysis/1Bxf...,178427,4
